# **DATA CLEANING.**

## **Libraries.**

In [1]:
import numpy as np
import pandas as pd
import re
import src_functions as ft

## **Dataset import.**

In [2]:
shark_attacks = pd.read_csv('../data/attacks.csv', encoding='unicode_escape')

shark_attacks

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Can see there is many NaN values in rows at the end of the table, so means there is null values in any column.**

## **Cleaning.**

### **Since there are many null values in the dataset in each column, we are going to eliminate all posible rows that have null values in all of it.**

In [3]:
shark_attacks.dropna(axis=0, inplace=True, how='all')

shark_attacks.shape

(8703, 24)

**We have gone from having about 26000 rows to having almost 9000. Now we are going to check the rows that are duplicates, and remove them.**

In [4]:
shark_attacks.duplicated().sum()

2392

In [5]:
shark_attacks = shark_attacks.drop_duplicates()
shark_attacks.reset_index(drop = True)

shark_attacks.shape

(6311, 24)

**Now, we have about 6000 rows, with hardly any null values in many of them, and without duplicates. Let's transform all Nan values.**

In [6]:
shark_attacks.fillna(0, inplace=True)

/var/folders/m0/lx952bgs23ddy9nzvwd3ms200000gn/T/ipykernel_5926/3791160861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks.fillna(0, inplace=True)


### **Now, we are going to transform certain columns, because there are many inconsistent values, and much of them are duplicated, or are null.**

In [7]:
# there are some columns that has a space in the name. Let's remove them
shark_attacks.columns = shark_attacks.columns.str.strip()

# we are going to replace the space between words in the columns by an underscore
shark_attacks = shark_attacks.rename(columns=lambda x: x.replace(' ', '_'))

shark_attacks.columns

Index(['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time',
       'Species', 'Investigator_or_Source', 'pdf', 'href_formula', 'href',
       'Case_Number.1', 'Case_Number.2', 'original_order', 'Unnamed:_22',
       'Unnamed:_23'],
      dtype='object')

#### **'Unnamed:_22' and 'Unnamed:_23'.**

In [8]:
shark_attacks['Unnamed:_22'].unique()

array([0, 'stopped here'], dtype=object)

In [9]:
shark_attacks['Unnamed:_23'].unique()

array([0, 'Teramo', 'change filename'], dtype=object)

**Both columns have mostly all null values. So let's fill it with the value 0.**

In [10]:
#renaming the columns
shark_attacks.rename(columns={'Unnamed:_22': 'Unnamed_1', 'Unnamed:_23': 'Unnamed_2'}, inplace=True)

# let's fill in all values in this columns

shark_attacks.loc[shark_attacks['Unnamed_1'] == 'stopped here', 'Unnamed_1'] = 0

shark_attacks.loc[shark_attacks['Unnamed_2'] == 'Teramo', 'Unnamed_2'] = 0
shark_attacks.loc[shark_attacks['Unnamed_2'] == 'change filename', 'Unnamed_2'] = 0

shark_attacks[['Unnamed_1', 'Unnamed_2']]

,Unnamed_1,Unnamed_2
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
6306,0,0
6307,0,0
6308,0,0
6309,0,0


In [11]:
shark_attacks['Unnamed_1'].unique()

array([0], dtype=object)

In [12]:
shark_attacks['Unnamed_2'].unique()

array([0], dtype=object)

#### **'Case_Number', 'Case_Number.1' and 'Case_Number.2'.**

In [13]:
shark_attacks['Case_Number'].unique()

array(['2018.06.25', '2018.06.18', '2018.06.09', ..., 'ND.0001', '0',
       'xx'], dtype=object)

In [14]:
shark_attacks['Case_Number.1'].unique()

array(['2018.06.25', '2018.06.18', '2018.06.09', ..., 'ND.0002',
       'ND.0001', 0], dtype=object)

In [15]:
shark_attacks['Case_Number.2'].unique()

array(['2018.06.25', '2018.06.18', '2018.06.09', ..., 'ND.0002',
       'ND.0001', 0], dtype=object)

In [16]:
shark_attacks[['Case_Number', 'Case_Number.1', 'Case_Number.2']]

,Case_Number,Case_Number.1,Case_Number.2
0,2018.06.25,2018.06.25,2018.06.25
1,2018.06.18,2018.06.18,2018.06.18
2,2018.06.09,2018.06.09,2018.06.09
3,2018.06.08,2018.06.08,2018.06.08
4,2018.06.04,2018.06.04,2018.06.04
...,...,...,...
6306,0,0,0
6307,0,0,0
6308,0,0,0
6309,0,0,0


**These three columns contain at first sight the same values, therefore we proceed to fill two of them with same values.**

In [17]:
#renaming the columns
shark_attacks.rename(columns={'Case_Number.1': 'NotCase_Number_1', 'Case_Number.2': 'NotCase_Number_2'}, inplace=True)

# let's fill in all values in 'NotCase_Number_1' and 'NotCase_Number_2'
shark_attacks['NotCase_Number_1'].fillna('useless', inplace=True)
shark_attacks['NotCase_Number_1'].replace(to_replace=shark_attacks['NotCase_Number_1'].unique(), value='useless', inplace=True)

shark_attacks['NotCase_Number_2'].fillna('useless', inplace=True)
shark_attacks['NotCase_Number_2'].replace(to_replace=shark_attacks['NotCase_Number_2'].unique(), value='useless', inplace=True)

# filling in NaN values in 'Case_Number'
shark_attacks['Case_Number'].fillna('no_number', inplace=True)

shark_attacks[['Case_Number', 'NotCase_Number_1', 'NotCase_Number_2']]

,Case_Number,NotCase_Number_1,NotCase_Number_2
0,2018.06.25,useless,useless
1,2018.06.18,useless,useless
2,2018.06.09,useless,useless
3,2018.06.08,useless,useless
4,2018.06.04,useless,useless
...,...,...,...
6306,0,useless,useless
6307,0,useless,useless
6308,0,useless,useless
6309,0,useless,useless


In [18]:
shark_attacks['NotCase_Number_1'].unique()

array(['useless'], dtype=object)

In [19]:
shark_attacks['NotCase_Number_2'].unique()

array(['useless'], dtype=object)

#### **'Year' and 'original_order'.**

In [20]:
shark_attacks['Year'].unique()

array([2018., 2017.,    0., 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [21]:
shark_attacks['original_order'].unique()

array([6303., 6302., 6301., ..., 6309., 6310.,    0.])

**We will transform the data to an integer, since they are float in both of them.**

In [22]:
# change the name of the column
shark_attacks = shark_attacks.rename(columns={'original_order': 'Original_order'})

# we transform the values
shark_attacks['Year'] = shark_attacks['Year'].astype(str).str[:-2].astype(int)
shark_attacks['Original_order'] = shark_attacks['Original_order'].astype(str).str[:-2].astype(int)

shark_attacks[['Year', 'Original_order']]

,Year,Original_order
0,2018,6303
1,2018,6302
2,2018,6301
3,2018,6300
4,2018,6299
...,...,...
6306,0,6308
6307,0,6309
6308,0,6310
6309,0,0


#### **'Activity'.**

In [23]:
shark_attacks['Activity'].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

**We are going to transform the values of the column that contain the main activities, by only that word.**

In [24]:
def replace_values(col):
    regex = re.compile(r"\b\w+ing\b")
    return col.str.replace(regex, 'water_sports')

shark_attacks['Activity'] = replace_values(shark_attacks['Activity'])

shark_attacks['Activity'].fillna(0, inplace=True)

shark_attacks['Activity']

0             water_sports
1             water_sports
2             water_sports
3             water_sports
4        Free water_sports
               ...        
6306                     0
6307                     0
6308                     0
6309                     0
25722                    0
Name: Activity, Length: 6311, dtype: object

#### **'Sex'.**

In [25]:
shark_attacks['Sex'].unique()

array(['F', 'M', 0, 'M ', 'lli', 'N', '.'], dtype=object)

**We are going to unify the values 'M' and 'F', and replace the others with 'not_registered'.**

In [26]:
shark_attacks['Sex'] = shark_attacks['Sex'].replace({'M ': 'M', 'lli': 'not_registered', 'N': 'not_registered', '.': 'not_registered', 0: 'not_registered'})

shark_attacks['Sex']

0                     F
1                     F
2                     M
3                     M
4                     M
              ...      
6306     not_registered
6307     not_registered
6308     not_registered
6309     not_registered
25722    not_registered
Name: Sex, Length: 6311, dtype: object

In [27]:
shark_attacks

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_or_Source,pdf,href_formula,href,NotCase_Number_1,NotCase_Number_2,Original_order,Unnamed_1,Unnamed_2
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",water_sports,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,useless,useless,6303,0,0
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",water_sports,Adyson McNeely,F,...,0,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,useless,useless,6302,0,0
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",water_sports,John Denges,M,...,0,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,useless,useless,6301,0,0
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,water_sports,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,useless,useless,6300,0,0
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free water_sports,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,useless,useless,6299,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,0,0,0,0,0,0,0,0,0,not_registered,...,0,0,0,0,0,useless,useless,6308,0,0
6307,0,0,0,0,0,0,0,0,0,not_registered,...,0,0,0,0,0,useless,useless,6309,0,0
6308,0,0,0,0,0,0,0,0,0,not_registered,...,0,0,0,0,0,useless,useless,6310,0,0
6309,0,0,0,0,0,0,0,0,0,not_registered,...,0,0,0,0,0,useless,useless,0,0,0


## **Export clean dataset.**

### **Let's export a copy of the clean dataset for working on it for do a deep analysis.**

In [28]:
shark_attacks.to_csv('../data/shark_attacks_clean.csv', index = False)